In [5]:
import nltk
import gensim
import matplotlib
from multiprocessing import cpu_count
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import nltk 
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krodr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krodr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krodr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Part 1: Exploratory Analysis of data 

# Loading and Reading the dataset. Head of Data displayed in out display.

# https://docs.google.com/spreadsheets/d/1wG2uPhDLVFbkhh5yB2w0Y3jcb6rzsoiCrQmzGb3m-sc/edit#gid=917237722

# sheet_id = '1wG2uPhDLVFbkhh5yB2w0Y3jcb6rzsoiCrQmzGb3m-sc'

df = '1wG2uPhDLVFbkhh5yB2w0Y3jcb6rzsoiCrQmzGb3m-sc'

xls = pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{df}/export?format=xlsx")

df = pd.read_excel(xls, 'Aspiring seeking human resource', header = 0)

df.head()

,id,job_title,location,connection,fit
0,1.0,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85.0,NaN
1,2.0,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3.0,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44.0,NaN
3,4.0,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5.0,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    float64
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(2), object(3)
memory usage: 4.2+ KB


In [8]:
df.replace({'Ä°zmir, TÃ¼rkiye': 'İzmir, Türkiye', 'Kanada' : 'Canada', '500+ ':'500'}, inplace=True) 

In [9]:
#change the data type of 'connection' column to 'int64'
df['connection'] = df['connection'].astype('int64')

In [10]:
df.head()

,id,job_title,location,connection,fit
0,1.0,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2.0,Native English Teacher at EPIK (English Progra...,Canada,500,NaN
2,3.0,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4.0,People Development Coordinator at Ryan,"Denton, Texas",500,NaN
4,5.0,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500,NaN


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    float64
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    int64  
 4   fit         0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.2+ KB


In [12]:
df.shape

(104, 5)

In [13]:
df.job_title[4]

'Advisory Board Member at Celal Bayar University'

In [14]:
#1 preprocess job_title column only
from nltk.corpus import stopwords
job_titl = df.job_title.apply(gensim.utils.simple_preprocess)
job_titl

0      [bauer, college, of, business, graduate, magna...
1      [native, english, teacher, at, epik, english, ...
2             [aspiring, human, resources, professional]
3           [people, development, coordinator, at, ryan]
4      [advisory, board, member, at, celal, bayar, un...
                             ...                        
99     [aspiring, human, resources, manager, graduati...
100          [human, resources, generalist, at, loparex]
101    [business, intelligence, and, analytics, at, t...
102                [always, set, them, up, for, success]
103    [director, of, administration, at, excellence,...
Name: job_title, Length: 104, dtype: object

In [15]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords[:10]  #check the first 10 stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [16]:
#2 define function to remove stop words from each of the job descriptions
def remove_stopwords (tokenized_text):
    cleaned_text = [word for word in tokenized_text if word not in stopwords]
    return cleaned_text

In [17]:
#3 remove stopwords from each job description
for ind, job in enumerate(job_titl):
    job_titl[ind] = remove_stopwords(job_titl[ind])  

job_titl

0      [bauer, college, business, graduate, magna, cu...
1      [native, english, teacher, epik, english, prog...
2             [aspiring, human, resources, professional]
3               [people, development, coordinator, ryan]
4      [advisory, board, member, celal, bayar, univer...
                             ...                        
99     [aspiring, human, resources, manager, graduati...
100              [human, resources, generalist, loparex]
101       [business, intelligence, analytics, travelers]
102                               [always, set, success]
103      [director, administration, excellence, logging]
Name: job_title, Length: 104, dtype: object

In [18]:
# 4preprocess and remove stopwords for location column
location_name = df.location.apply(gensim.utils.simple_preprocess)

for ind, location in enumerate(location_name):
    location_name[ind] = remove_stopwords(location_name[ind])  
location_name

0                              [houston, texas]
1                                      [canada]
2      [raleigh, durham, north, carolina, area]
3                               [denton, texas]
4                               [zmir, türkiye]
                         ...                   
99                  [cape, girardeau, missouri]
100    [raleigh, durham, north, carolina, area]
101            [greater, new, york, city, area]
102               [greater, los, angeles, area]
103                               [katy, texas]
Name: location, Length: 104, dtype: object

In [19]:
#5 
job_location_combined = job_titl.append(location_name)
job_location_combined

0      [bauer, college, business, graduate, magna, cu...
1      [native, english, teacher, epik, english, prog...
2             [aspiring, human, resources, professional]
3               [people, development, coordinator, ryan]
4      [advisory, board, member, celal, bayar, univer...
                             ...                        
99                           [cape, girardeau, missouri]
100             [raleigh, durham, north, carolina, area]
101                     [greater, new, york, city, area]
102                        [greater, los, angeles, area]
103                                        [katy, texas]
Length: 208, dtype: object

In [20]:
cpu_count()

12

In [21]:
# now we initialize a gensim model
# window is how many words before and after the target word. Can be anything 5, 7 etc
# min_count is the min length of a sentense. So don't use sentences with only 1 word
# workers how many CPU threads to be used?? (so CPUs with 4 cores, use 4, my Lenovo laptop has 2 cores but supposedly
# can handle upto 4 threads in parallel.. whatever that means) 

model = gensim.models.Word2Vec(window = 5, min_count = 2, workers = 12)  # cpu_count for this laptop is 4

In [22]:
model.build_vocab(job_location_combined)

In [23]:
model.corpus_count, model.epochs

(208, 5)

In [24]:
model.train(job_location_combined, total_examples = model.corpus_count, epochs = model.epochs)

(1523, 4695)

In [25]:
key_words = remove_stopwords(gensim.utils.simple_preprocess('Aspiring seeking human resources'))
key_words

['aspiring', 'seeking', 'human', 'resources']

In [26]:
#define function to compute cosine similarity between two lists of words that are not individual
def word_set_similarity (test_set, reference):
    return model.wv.n_similarity (test_set, reference)

In [27]:
score = word_set_similarity(job_titl[5], key_words)
score

0.7476511

In [28]:
def rem_stopwrds_preprocess(sentence):
    return remove_stopwords(gensim.utils.simple_preprocess(sentence))

In [29]:
key_words = rem_stopwrds_preprocess('Aspiring seeking human resources') # enter 'seeking human resources' or  'Aspiring human resources'

# each candidate job description is compared with key words using the function to calculate cosine similarity
df['fit'] = df['job_title'].apply(lambda x: word_set_similarity(rem_stopwrds_preprocess(x), key_words))
candidates_1 = df.sort_values('fit', ignore_index=True, ascending=False)
candidates_1.rename(columns={'fit':'job_fit'}, inplace=True)
candidates_1 = candidates_1[candidates_1['job_fit']>0]

candidates_1

,id,job_title,location,connection,job_fit
0,73.0,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,0.885104
1,100.0,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,0.859920
2,27.0,Aspiring Human Resources Management student se...,"Houston, Texas Area",500,0.793262
3,29.0,Aspiring Human Resources Management student se...,"Houston, Texas Area",500,0.793262
4,28.0,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.773973
...,...,...,...,...,...
77,11.0,Student at Chapman University,"Lake Forest, California",2,0.071705
78,54.0,Student at Chapman University,"Lake Forest, California",2,0.071705
79,63.0,Student at Chapman University,"Lake Forest, California",2,0.071705
80,102.0,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,0.042597


In [30]:
rem_stopwrds_preprocess(df[df.id==102].iloc[0]['location'])

['greater', 'new', 'york', 'city', 'area']

In [31]:
 # following function takes the id number of the preferred candidate from the first round and uses that candidate's
# location to rerank the candidates based on the job_title AND location. For the ranking, job title is given the highest weight,
# followed by candidate location followed by the candidate's number of LinkedIn connections

def rerank (id_num, input_df):
    '''
    input: id_num - id number of the preferred candidate from the first round of screening
    input: input_df - this is the df where the data from the first sceening round is stored
    
    output: df containing the new columns where job and location similarity values, a score for the number of
            connections as well as a column for the average of the weighted scores.
    '''
    # remove the previously created columns in the input df since they will be calculated again in this function
    df = input_df.iloc[:, :4]
    
    #preferred candidate job_title embedding
    cand_job_preprocess = rem_stopwrds_preprocess (df[df.id==id_num].iloc[0]['job_title'])
    #preferred candidate location embedding
    cand_loc_preprocess = rem_stopwrds_preprocess (df[df.id==id_num].iloc[0]['location'])
    
    # calculate cosine similarities of the job fit for each candidate based on the job description of the preferred candidate
    df['job_fit'] = df['job_title'].apply(lambda x: word_set_similarity(rem_stopwrds_preprocess(x), cand_job_preprocess))
    # calculate cosine similarities of the location fit for each candidate based on the job description of the preferred candidate
    df['location_fit'] = df['location'].apply(lambda x: word_set_similarity(rem_stopwrds_preprocess(x), cand_loc_preprocess))
    
    # create a column for connection count and then normalize to the maximum value in the column
    df['connection_score'] = df['connection'].apply(lambda x: x/df['connection'].max()) 

    #calculate the mean for the job_fit and location_fit and conection_score values
    # BUT give the job_fit more weigt (*3) followed by the location_fit (*2) and then connection_score(*0.5)
    df['job_loc_connection_fit'] = (df['job_fit']*3+df['location_fit']*2+df['connection_score']*0.5)/3
    
    #sort the df based on the mean of weighted scores 
    df = df.sort_values('job_loc_connection_fit', ignore_index = True, ascending=False)
    
    return df

In [32]:
# rerank the list based on candidate id 53
output_df = rerank(53, candidates_1)
output_df.head(10)

,id,job_title,location,connection,job_fit,location_fit,connection_score,job_loc_connection_fit
0,53.0,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500,1.000000,1.000000,1.000,1.833333
1,62.0,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500,1.000000,1.000000,1.000,1.833333
2,10.0,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500,1.000000,1.000000,1.000,1.833333
3,40.0,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500,1.000000,1.000000,1.000,1.833333
4,69.0,"Director of Human Resources North America, Gro...","Greater Grand Rapids, Michigan Area",500,0.391234,0.817515,1.000,1.102910
5,74.0,Human Resources Professional,Greater Boston Area,16,0.516438,0.817515,0.032,1.066781
6,89.0,Director Human Resources at EY,Greater Atlanta Area,349,0.434369,0.718418,0.698,1.029648
7,99.0,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,0.603871,0.583191,0.096,1.008665
8,68.0,Human Resources Specialist at Luxottica,Greater New York City Area,500,0.511840,0.482865,1.000,1.000417
9,27.0,Aspiring Human Resources Management student se...,"Houston, Texas Area",500,0.552762,0.411324,1.000,0.993644
